In [73]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import matplotlib.pylab as pl
import numpy as np

In [74]:
class SupportVectorMachine(object):
    """
    Linear Support Vector Machine
    """
    def __init__(self, C=1.0,):
        self.C = C
        
    def predict(self, X):
        """
        Predicts labels for the provided data: {-1, 1}
        Args:
            X: dataframe that contains data. Shape (m, n)
        """
        return np.sign(np.dot(X, self.w) + self.b) 
    
    def fit(self, X, y):
        """
        Args:
            X: data with m samples and n features. Shape (m, n)
            y: 
        """
        m, n = X.shape
        y = y.reshape(-1, 1) 
        prod = y * X 
        H =  np.dot(prod, prod.T)
        
        # Quadratic Progrmaming to solve for Lagrange Multipliers
        P = cvxopt_matrix(H)
        q = cvxopt_matrix(-np.ones((m, 1)))
        # 2D array with ones on the diagonal and zeros elsewhere
        G = cvxopt_matrix(np.vstack((np.eye(m)* -1, np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt_matrix(y.reshape(1, -1))
        b = cvxopt_matrix(np.zeros(1))
        
        cvxopt_solvers.options['show_progress'] = False
        cvxopt_solvers.options['abstol'] = 1e-10
        cvxopt_solvers.options['reltol'] = 1e-10
        cvxopt_solvers.options['feastol'] = 1e-10
        
        sol = cvxopt_solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])
        
        # Compute W
        self.w = ((y * alphas).T @ X).reshape(-1, 1)
        
        # Support Vectors
        S = (alphas > 1e-4).flatten() 
        
        # Computer b
        self.b = y[S] - np.dot(X[S], self.w)
        

In [75]:
def gen_lin_separable_data():
    # generate training data in the 2-d case
    mean1 = np.array([0, 2])
    mean2 = np.array([2, 0])
    cov = np.array([[0.8, 0.6], [0.6, 0.8]] )
    X1 = np.random.multivariate_normal(mean1, cov, 100)
    y1 = np.ones(len(X1))
    X2 = np.random.multivariate_normal(mean2, cov, 100)
    y2 = np.ones(len(X2)) * -1
    return X1, y1, X2, y2

In [76]:
X1, y1, X2, y2 = gen_non_lin_separable_data()

In [77]:
clf = SupportVectorMachine()

In [78]:
clf.fit(X, y)

TypeError: 'P' must be a 'd' matrix of size (7, 7)

In [ ]:
clf.w

In [ ]:
clf.b